# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

In [67]:
#from google.colab import drive
#drive.mount('/content/drive')

In [68]:
#import pandas as pd
#import numpy as np

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [69]:
df = pd.read_csv('/content/drive/MyDrive/application_record.csv')

In [70]:
mau = pd.read_csv('/content/drive/MyDrive/credit_record.csv')

In [71]:
print(df.shape)
print(mau.shape)

(438557, 18)
(1048575, 3)


In [72]:
df.head(2)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0


**LIMPEZA**

In [73]:
#================= MISSINGS==============
print(mau.isna().sum())

ID                0
MONTHS_BALANCE    0
STATUS            0
dtype: int64


In [74]:
display(df.isna().sum().sum())

134203

In [75]:
#============= COLUNAS DESNECESSÁRIAS E COM MISSINGS=====================

df.drop(columns=['NAME_HOUSING_TYPE'],inplace=True)
df.drop(columns=['OCCUPATION_TYPE'],inplace=True)

**IDENTIFICANDO ID DUPLICADOS**

In [76]:
print(df.ID.duplicated().sum())
print(mau.duplicated(subset=['ID']).sum())

47
1002590


In [77]:
#========== LIMPEZA DE IDs DUPLICADOS===================

In [78]:
df.drop_duplicates(subset='ID',inplace=True)
mau.drop_duplicates(subset='ID',inplace=True)

In [ ]:
df.info()

In [80]:
mau.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45985 entries, 0 to 1048545
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              45985 non-null  int64 
 1   MONTHS_BALANCE  45985 non-null  int64 
 2   STATUS          45985 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.4+ MB


**CATEGORIZANDO VARIÁVEIS**

In [81]:
df.loc[df.CODE_GENDER == 'M','CODE_GENDER']=0
df.loc[df.CODE_GENDER == 'F','CODE_GENDER']=1
df.loc[df.FLAG_OWN_CAR == 'N','FLAG_OWN_CAR']=0
df.loc[df.FLAG_OWN_CAR == 'Y','FLAG_OWN_CAR']=1
df.loc[df.FLAG_OWN_REALTY == 'N','FLAG_OWN_REALTY']=0
df.loc[df.FLAG_OWN_REALTY == 'Y','FLAG_OWN_REALTY']=1

In [82]:
civil = ['Married','Single / not married','Civil marriage','Separated','Widow']
n=0
for item in civil:
  df.loc[df.NAME_FAMILY_STATUS == item,'NAME_FAMILY_STATUS']=n
  n+=1

study = ['Secondary / secondary special','Higher education','Incomplete higher','Lower secondary','Academic degree']
n=0
for item in study:
  df.loc[df.NAME_EDUCATION_TYPE == item,'NAME_EDUCATION_TYPE']=n
  n+=1

work = ['Working','Commercial associate','Pensioner','State servant', 'Student']
n=0
for item in work:
  df.loc[df.NAME_INCOME_TYPE == item,'NAME_INCOME_TYPE']=n
  n+=1

**TIPOS DE DADOS**

In [83]:
#==================== CONVERSÃO DE COLUNAS=======================
df['CNT_FAM_MEMBERS'] = df['CNT_FAM_MEMBERS'].astype(int)
df['CODE_GENDER'] = df['CODE_GENDER'].astype(int)
df['FLAG_OWN_CAR'] = df['FLAG_OWN_CAR'].astype(int)
df['FLAG_OWN_REALTY'] = df['FLAG_OWN_REALTY'].astype(int)
df['NAME_INCOME_TYPE'] = df['NAME_INCOME_TYPE'].astype(int)
df['NAME_EDUCATION_TYPE'] = df['NAME_EDUCATION_TYPE'].astype(int)
df['NAME_FAMILY_STATUS'] = df['NAME_FAMILY_STATUS'].astype(int)

**UNINDO AS TABELAS df E mau**

In [84]:
df1 = df.merge(mau,on='ID',how='left')

**LIMPEZA DO NOVO DATAFRAME**

In [ ]:
df1.isna().sum()

In [86]:
df1.dropna(how='any',inplace=True)
df1.drop(columns=['MONTHS_BALANCE'],inplace=True)

**CONVERTENDO VALORES DA COLUNA STATUS**

In [ ]:
df1.STATUS.value_counts()

In [88]:
df1.loc[df1.STATUS == 'C','STATUS']=7
df1.loc[df1.STATUS == 'X','STATUS']=6

In [ ]:
df1.info()

In [90]:
df1.STATUS = df1.STATUS.astype(int)

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [91]:
previsao = pd.crosstab(df1.FLAG_EMAIL, df1.FLAG_OWN_CAR)

In [92]:
previsao

FLAG_OWN_CAR,0,1
FLAG_EMAIL,,
0,20695,12491
1,1919,1352


In [93]:
perc = previsao.div(previsao.sum(axis=1),axis=0)

In [94]:
perc

FLAG_OWN_CAR,0,1
FLAG_EMAIL,,
0,0.623606,0.376394
1,0.586671,0.413329


#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [95]:
df1.reset_index(inplace=True)

In [96]:
df1.head()

,index,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,STATUS
0,0,5008804,0,1,1,0,427500.0,0,1,2,-12005,-4542,1,1,0,0,2,7
1,1,5008805,0,1,1,0,427500.0,0,1,2,-12005,-4542,1,1,0,0,2,7
2,2,5008806,0,1,1,0,112500.0,0,0,0,-21474,-1134,1,0,0,0,2,7
3,3,5008808,1,0,1,0,270000.0,1,0,1,-19110,-3051,1,0,1,1,1,0
4,4,5008809,1,0,1,0,270000.0,1,0,1,-19110,-3051,1,0,1,1,1,6


In [97]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36457 entries, 0 to 36456
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                36457 non-null  int64  
 1   ID                   36457 non-null  int64  
 2   CODE_GENDER          36457 non-null  int64  
 3   FLAG_OWN_CAR         36457 non-null  int64  
 4   FLAG_OWN_REALTY      36457 non-null  int64  
 5   CNT_CHILDREN         36457 non-null  int64  
 6   AMT_INCOME_TOTAL     36457 non-null  float64
 7   NAME_INCOME_TYPE     36457 non-null  int64  
 8   NAME_EDUCATION_TYPE  36457 non-null  int64  
 9   NAME_FAMILY_STATUS   36457 non-null  int64  
 10  DAYS_BIRTH           36457 non-null  int64  
 11  DAYS_EMPLOYED        36457 non-null  int64  
 12  FLAG_MOBIL           36457 non-null  int64  
 13  FLAG_WORK_PHONE      36457 non-null  int64  
 14  FLAG_PHONE           36457 non-null  int64  
 15  FLAG_EMAIL           36457 non-null 